In [82]:
# Cleaning data

#!pip install scikit-learn
from sklearn.model_selection import train_test_split

#!pip install pandas
import pandas as pd 

mythenquai = pd.read_csv('./input/messwerte_mythenquai_2007-2021.csv')
tiefenbrunnen = pd.read_csv('./input/messwerte_tiefenbrunnen_2007-2021.csv')

mythenquai['station'] = 'mythenquai'
tiefenbrunnen['station'] = 'tiefenbrunnen'

data = pd.concat([mythenquai, tiefenbrunnen])

# drop timestamp_utc, global_radiation, water_level and na values
data = data.drop(columns=['timestamp_utc', 'global_radiation', 'water_level'], axis=1)
data = data.dropna(how='any')

In [93]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
data['timestamp_cet'] = le.fit_transform(data['timestamp_cet'])
data['station'] = le.fit_transform(data['station'])

Y = data['air_temperature']
X = data.drop(columns=['air_temperature'])

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=9)

# defining the model
model = KNeighborsRegressor(n_neighbors=60,n_jobs=-1)

# training model

model.fit(X_train, Y_train)

# predicting target values
Y_predict6 = model.predict(X_test)

In [94]:
# test accuracy of trained model
test_acc_model = round(model.fit(X_train,Y_train).score(X_test, Y_test)* 100, 2)
train_acc_model = round(model.fit(X_train, Y_train).score(X_train, Y_train)* 100, 2)

model1 = pd.DataFrame({
    'Model': ['KNN'],
    'Train Score': [train_acc_model],
    'Test Score': [test_acc_model]
})

model1.sort_values(by='Test Score', ascending=False)

,Model,Train Score,Test Score
0,KNN,92.22,91.91


In [169]:
# making a prediction for given day
cols = ['timestamp_cet','water_temperature','wind_gust_max_10min','wind_speed_avg_10min','wind_force_avg_10min','wind_direction','windchill','barometric_pressure_qfe','precipitation','dew_point','humidity','station']

input1 = pd.read_csv('./input/input.csv')
input1['station'] = 'mythenquai'
input1 = input1.drop(columns=['timestamp_utc', 'global_radiation', 'water_level','air_temperature'], axis=1)
input1['timestamp_cet'] = le.fit_transform(input1['timestamp_cet'])
input1['station'] = le.fit_transform(input1['station'])

Y_prediction = model.predict(input1)

Y_prediction

array([18.62666667])